In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('Data.csv')
df = df.drop('Unnamed: 0', axis=1)

train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

df = train

X = df[['a','b']]
y = df[['c','d']]

X=X.to_numpy()
y=y.to_numpy()

lr = 0.001
momentum = 0.001
maximum = df.max(axis=0) # will return max value of each column
minimum = df.min(axis=0) # will return max value of each column

print(maximum,minimum)
len(test)

D:\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


a     810.270083
b    1024.800000
c       8.000000
d       7.941556
dtype: float64 a   -791.132347
b     65.002541
c     -7.700000
d     -7.913990
dtype: float64


28306

In [2]:
def norm(x,maximum,minimum):
    y = ((x-minimum)/(maximum-minimum))
    return y

def denorm(x,maximum,minimum):
    y = (x*(maximum-minimum)+minimum)
    return y
print("BEFORE NORMALIZEATION",y[:,1])
X[:,0] = norm(X[:,0],maximum[0],minimum[0])
X[:,1] = norm(X[:,1],maximum[1],minimum[1])
y[:,0] = norm(y[:,0],maximum[2],minimum[2])
y[:,1] = norm(y[:,1],maximum[3],minimum[3])
print("AFTER NORMALIZEATION",y[:,1])
print("AFTER DENORMALIZEATION",denorm(y[:,1],maximum[3],minimum[3]))

BEFORE NORMALIZEATION [ 0.01511941  2.53939614 -0.01100439 ...  0.01184836 -1.17219779
  2.54752167]
AFTER NORMALIZEATION [0.5000843  0.65928895 0.49843668 ... 0.49987799 0.4252009  0.65980142]
AFTER DENORMALIZEATION [ 0.01511941  2.53939614 -0.01100439 ...  0.01184836 -1.17219779
  2.54752167]


In [3]:
class NeuralNetwork(object):
    def __init__(self):
        self.input = 2
        self.output = 2
        self.hidden = 4
        
        self.W1 = np.random.rand(self.input, self.hidden) 
        self.W2 = np.random.rand(self.hidden, self.output)

    def ForwardProp(self, X):
        self.weight = np.dot(X, self.W1)
        self.weight2 = self.sigmoidActivation(self.weight) 
        self.weight3 = np.dot(self.weight2, self.W2) 
        output = self.sigmoidActivation(self.weight3)
        return output
        
    def sigmoidActivation(self, s, deriv=False):
        if (deriv == True):
            return s * (1 - s)
        return 1/(1 + np.exp(-s))
    
    def backwardProp(self, X, y, output):
        #backward propogate through the network
        self.output_error = y - output # error in output
        self.output_delta = self.output_error * self.sigmoidActivation(output, deriv=True)
        
        self.z2_error = self.output_delta.dot(self.W2.T) #z2 error: how much our hidden layer weights contribute to output error
        self.z2_delta = self.z2_error * self.sigmoidActivation(self.weight2, deriv=True) #applying derivative of sigmoid to z2 error
        
        self.W1 += X.T.dot(self.z2_delta)*lr + momentum * self.W1 # adjusting first set (input -> hidden) weights
        self.W2 += self.weight2.T.dot(self.output_delta)*lr + momentum * self.W2 # adjusting second set (hidden -> output) weights
        
    def train(self, X, y):
        output = self.ForwardProp(X)
        self.backwardProp(X, y, output)



In [ ]:
NN = NeuralNetwork()
RMSE = []
MSE = []

i=0
while True: #trains the NN 1000 times
    i=i+1
    RMSE.append(str(np.sqrt(((y - NN.ForwardProp(X))**2).mean())))
    MSE.append(str(np.mean(np.square(y - NN.ForwardProp(X)))))

    if (i % 100 == 0 or i == 1): 
        print("")
        print("MSE in epochs",str(i)+": " + str(np.mean(np.square(y - NN.ForwardProp(X)))))
        print("RMSE", np.sqrt(((y - NN.ForwardProp(X))**2).mean()))
        print("W1",NN.W1)
        print("")
        print("W2",NN.W2)
        print("")

    #len(error) > 1 and round(float(error[-1]),11)== round(float(error[-2]),11) or 
    if(len(RMSE) > 1 and round(float(RMSE[-1]),6)== round(float(RMSE[-2]),6)):
        print("Loss: same in previous epoch, stopping the training at epoch number",i)
        print("")
        print("MSE in epochs",str(i)+": " + str(np.mean(np.square(y - NN.ForwardProp(X)))))
        print("RMSE", np.sqrt(((y - NN.ForwardProp(X))**2).mean()))
        print("W1",NN.W1)
        print("")
        print("W2",NN.W2)
        print("")
        break
    else:
        NN.train(X, y)



MSE in epochs 1: 0.09858844838901487
RMSE 0.3139879749114843
W1 [[0.87793519 0.21475107 0.57425822 0.55283247]
 [0.3513477  0.7557816  0.66223591 0.35442225]]

W2 [[0.93304352 0.96740521]
 [0.55051134 0.4856535 ]
 [0.82686357 0.46540704]
 [0.32565609 0.15972775]]


MSE in epochs 100: 0.008308098671162232
RMSE 0.09114877218680585
W1 [[-2.09707439 -4.16296213 -3.78662669 -3.67964481]
 [-3.06260721 -1.54301987 -2.41300084 -2.70050073]]

W2 [[ 1.06228174  0.85832379]
 [-0.67821637 -0.79369894]
 [ 0.2094177  -0.56962548]
 [ 0.42896598 -0.53114255]]



In [ ]:
import matplotlib.pyplot as plt
 
df = pd.DataFrame({'RMSE':RMSE})

df = df.astype(float)
df.plot(figsize=(10,3))

In [ ]:
df = test

X = df[['a','b']]
y = df[['c','d']]

X=X.to_numpy()
y=y.to_numpy()

print("BEFORE NORMALIZEATION",y[:,1])
y[:,0] = norm(y[:,0],maximum[2],minimum[2])
y[:,1] = norm(y[:,1],maximum[3],minimum[3])
print("AFTER NORMALIZEATION",y[:,1])
print("AFTER DENORMALIZEATION",denorm(y[:,1],maximum[3],minimum[3]))

i=75

output = NN.ForwardProp(X[i])
output[0] = denorm(output[0],maximum[2],minimum[2])
output[1] = denorm(output[1],maximum[3],minimum[3])

y[i][0] = denorm(y[i][0],maximum[2],minimum[2])
y[i][1] = denorm(y[i][1],maximum[3],minimum[3])


inputt = X[i]
inputt[0] = norm(output[0],maximum[0],minimum[0])
inputt[1] = norm(output[1],maximum[1],minimum[1])

In [ ]:
print("\n")        
print("Input: " + str([X[i]]))
print("\n")     
print("Actual Output: " + str(y[i]))
print("RMSE: " +str(np.sqrt(((y - NN.ForwardProp(X))**2).mean())))
print("\n")
print("Predicted Output: " , output)